<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [23]:
#Importo las librerías requeridas
!pip install pandas
import pandas as pd
import tweepy
import re 
import regex
import emoji
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [36]:
api_key = "Ingreso_seguro"
api_secret_key = "Credenciales_user"
access_token = "de_uso_propio"
access_token_secret =  "personales"

In [37]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [38]:
tweets = tweepy.Cursor(api.search,
                       q=['#XboxGamePassUltimate -RT'],
                       lang='es',
                       # geocode='4.7110,-74.0721,100km', #Filtra por localizacion 
                       tweet_mode='extended',# Para que llegue el texto completo
                       include_rts=False, # Para evitar que sea re tuits
                       ).items(100)

In [39]:

# Convirte la informacion en una lista
tuits = [tweet for tweet in tweets]

TweepError: Twitter error response: status code = 401

In [30]:
tuits[0].full_text

NameError: name 'tuits' is not defined

In [31]:
tweet = []

for tuit in tuits: 
    temp = []
    temp.append(tuit.full_text)
    temp.append(tuit.created_at)
    temp.append(tuit.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tuit.id}")
    
    tweet.append(temp)
    
    
data = pd.DataFrame(tweet)
data.columns = ['tweets', 'date', 'id', 'link']
data.head()


NameError: name 'tuits' is not defined

In [32]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [33]:
#Obtiene el texto limpio, acortamos la url, obtenemos los hashtags y las cuentas mencionadas
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

NameError: name 'data' is not defined

In [34]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

In [35]:
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

NameError: name 'data' is not defined

###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [ ]:

data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')

In [ ]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora para #XboxGamePassUltimate")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)